In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D,MaxPool2D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.python.client import device_lib
import itertools
import os 
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
from tensorflow.python.client import device_lib
import sklearn
import wandb
from wandb.keras import WandbCallback
import glob


In [6]:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
run = wandb.init(project='resnet50',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.0001,
                     "epochs": 40,
                     "batch_size": 2,
                     "loss_function": "categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "malwareDB"
                 })
config = wandb.config

In [8]:
train_path = 'C:/Users/Kacper/Desktop/inzynier/code/data/legit_vs_malware/train'
valid_path = 'C:/Users/Kacper/Desktop/inzynier/code/data/legit_vs_malware/valid'
test_path = 'C:/Users/Kacper/Desktop/inzynier/code/data/legit_vs_malware/test'

#1153,4612

In [9]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224),classes=['cerberus','hydra','alien','other','legit'],batch_size=2)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224),classes=['cerberus','hydra','alien','other','legit'],batch_size=2)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224),classes=['cerberus','hydra','alien','other','legit'],batch_size=2,shuffle=False)


Found 994 images belonging to 5 classes.
Found 141 images belonging to 5 classes.
Found 151 images belonging to 5 classes.


In [10]:
assert train_batches.n == 994
assert valid_batches.n == 141
assert test_batches.n == 151
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 5

In [11]:
cm_plot_labels = ['cerberus','hydra','alien','other','legit']

In [12]:
def calculateF1(cm):
    precision = 0
    recall = 0
    TP = cm[4][4]
    FP = 0
    FN = 0
    TN = 0
    for i in range(0,4):
        FP = FP + cm[i][4]
    for i in range (0,4):
        FN = FN + cm[4][i]
    for i in cm:
        for j in i:
            TN = TN + j
    TN = TN - (FP + FN + TP)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    return 2*((precision*recall)/(precision+recall))

In [13]:
base_model = keras.applications.resnet50.ResNet50(include_top=False)

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)

predictions = Dense(5, activation='softmax')(x)

model = keras.models.Model(inputs=base_model.input, outputs=predictions)

In [14]:
for layer in base_model.layers:
    layer.trainable = False

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics="accuracy")

In [17]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=40,verbose=2,callbacks=[WandbCallback(data_type="image", validation_data=valid_batches, labels=cm_plot_labels)])

Epoch 1/40


C:\Users\Kacper\.conda\envs\test\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


497/497 - 17s - loss: 0.5945 - accuracy: 0.8169 - val_loss: 0.8014 - val_accuracy: 0.6738 - 17s/epoch - 34ms/step
Epoch 2/40
497/497 - 12s - loss: 0.3676 - accuracy: 0.8652 - val_loss: 0.4051 - val_accuracy: 0.8652 - 12s/epoch - 25ms/step
Epoch 3/40
497/497 - 12s - loss: 0.2680 - accuracy: 0.8984 - val_loss: 0.4911 - val_accuracy: 0.7943 - 12s/epoch - 24ms/step
Epoch 4/40
497/497 - 12s - loss: 0.2260 - accuracy: 0.9185 - val_loss: 0.7013 - val_accuracy: 0.7518 - 12s/epoch - 24ms/step
Epoch 5/40
497/497 - 12s - loss: 0.1924 - accuracy: 0.9306 - val_loss: 0.6609 - val_accuracy: 0.8227 - 12s/epoch - 25ms/step
Epoch 6/40
497/497 - 12s - loss: 0.1568 - accuracy: 0.9386 - val_loss: 0.5236 - val_accuracy: 0.8014 - 12s/epoch - 24ms/step
Epoch 7/40
497/497 - 12s - loss: 0.1240 - accuracy: 0.9497 - val_loss: 0.4459 - val_accuracy: 0.8723 - 12s/epoch - 24ms/step
Epoch 8/40
497/497 - 12s - loss: 0.1373 - accuracy: 0.9447 - val_loss: 0.4688 - val_accuracy: 0.8723 - 12s/epoch - 24ms/step
Epoch 9/40


In [18]:
predictions = model.predict(x=test_batches, verbose=0)

In [19]:
cm = confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(predictions,axis=-1))

In [20]:
print(calculateF1(cm))

0.9714285714285714


In [21]:
wandb.log({
            "F1_score" : calculateF1(cm),
            "confusion_matrix" : wandb.sklearn.plot_confusion_matrix(test_batches.classes,
                                                            np.argmax(predictions,axis=-1),
                                                            cm_plot_labels)
            })

In [22]:
wandb.finish()

F1_score,▁
accuracy,▁▃▄▅▅▆▆▆▇▆▇▇▇█▇▇▇▇▇███████▇██▇████▇█████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▂▁▁▁▁▁
val_accuracy,▁▇▅▃▅▅▇▇▇▇▆▆▇▇▆▇▄▇▆▇█▇▇▇▇▇█▆█▇▇█▇▇▆▇▇▇▇█
val_loss,▇▁▂▆▅▃▂▂▃▁▃▃▃▃▃▃█▃▃▃▂▄▃▂▃▃▃▆▄▄▃▄▄▄█▄▄▄▄▄
F1_score,0.97143
accuracy,1.0
best_epoch,1
best_val_loss,0.4051
epoch,39
